In [ ]:
# 01. Getting Started with Kubiya Workflow SDK

This notebook demonstrates the basics of the Kubiya Workflow SDK, including:
- Creating simple workflows
- Using different executors
- Working with parameters
- Error handling
- Viewing execution outputs

**Note**: The SDK will automatically select an appropriate runner. You can also configure local Kubernetes runners for advanced use cases.


In [ ]:
# Setup and imports
import os
import sys
from dotenv import load_dotenv

# First, check if we're in the right environment
print("🔍 Checking environment setup...")

# Verify Python version
if sys.version_info < (3, 8):
    print(f"❌ Python {sys.version_info.major}.{sys.version_info.minor} is too old. Please use Python 3.8+")
    raise SystemError("Python 3.8+ required")

# Try to import the SDK
try:
    from kubiya_workflow_sdk import workflow, step, execute_workflow
    from kubiya_workflow_sdk.client import KubiyaClient
    import json
    print("✅ Kubiya SDK imported successfully")
except ImportError:
    print("❌ Kubiya SDK not installed!")
    print("   Please run: pip install kubiya-workflow-sdk")
    print("   Or go back to 00_test_setup.ipynb")
    raise

# Load environment variables
load_dotenv()

# Configuration
API_KEY = os.getenv("KUBIYA_API_KEY")

if not API_KEY:
    print("\n❌ KUBIYA_API_KEY not set!")
    print("   1. Create a .env file in this directory")
    print("   2. Add: KUBIYA_API_KEY=your-api-key")
    print("   3. Get your key from: https://app.kubiya.ai/settings/api-keys")
    print("\n   Or run 00_test_setup.ipynb for guided setup")
    raise ValueError("API key required")
else:
    print(f"✅ API Key loaded (length: {len(API_KEY)})")
    print("🔗 Using Kubiya API at: https://api.kubiya.ai")
    print("📋 The SDK will automatically select an appropriate runner")
    print("\n✅ Ready to start!")


In [ ]:
## 1. Basic Workflow Creation

Let's start with a simple "Hello World" workflow that demonstrates the basic structure:


# Create a simple workflow
hello_workflow = (
    workflow("hello-world")
    .description("My first Kubiya workflow")
    .step("greeting", "echo '🎉 Hello from Kubiya SDK!'")
    .step("timestamp", "date '+%Y-%m-%d %H:%M:%S'")
    .step("complete", "echo '✅ Workflow execution complete!'")
)

# Display the workflow definition
print("📋 Workflow created:")
print(f"   Name: {hello_workflow.data['name']}")
print(f"   Steps: {[s['name'] for s in hello_workflow.data['steps']]}")


In [ ]:
# Execute the workflow and capture outputs
print("\n🚀 Executing workflow...")
print("=" * 60)

outputs = []
step_count = 0

try:
    for event in execute_workflow(
        workflow_definition=hello_workflow.to_dict(),
        api_key=API_KEY,
        stream=True
    ):
        # Parse event data
        try:
            event_data = json.loads(event)
            
            # Capture outputs
            if 'output' in event_data:
                output = event_data['output'].strip()
                if output:
                    outputs.append(output)
                    print(f"📤 Output: {output}")
            
            # Track step completion
            elif event_data.get('type') == 'step_finished':
                step_count += 1
                step_name = event_data.get('step', {}).get('name', 'unknown')
                print(f"✅ Completed step: {step_name}")
        except:
            pass
    
    print("=" * 60)
    print(f"✅ Workflow completed successfully!")
    print(f"   Steps executed: {step_count}")
    print(f"   Outputs received: {len(outputs)}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    print("\n💡 Troubleshooting tips:")
    print("   1. Verify your API key is valid")
    print("   2. Check network connectivity")
    print("   3. Try again in a few moments")


In [ ]:
## 2. Parameterized Workflows

Workflows can accept parameters that are passed at execution time:


In [ ]:
# Create a parameterized workflow
greeting_workflow = (
    workflow("personalized-greeting")
    .description("Workflow with parameters")
    .params(
        name="${NAME:-World}",
        language="${LANGUAGE:-English}",
        count="${COUNT:-1}"
    )
    .step("greet", """
        case "${language}" in
            "Spanish") greeting="¡Hola" ;;
            "French") greeting="Bonjour" ;;
            "German") greeting="Guten Tag" ;;
            *) greeting="Hello" ;;
        esac
        
        for i in $(seq 1 ${count}); do
            echo "$i. $greeting, ${name}! 👋"
        done
    """)
    .step("timestamp", "echo 'Generated at:' $(date '+%Y-%m-%d %H:%M:%S')")
)

print("📋 Created parameterized workflow with:")
print(f"   Parameters: name, language, count")
print(f"   Default values: World, English, 1")


In [ ]:
# Execute with custom parameters
params = {
    "name": "Kubiya User",
    "language": "Spanish", 
    "count": "3"
}

print(f"\n🚀 Executing with parameters: {params}")
print("=" * 60)

try:
    for event in execute_workflow(
        workflow_definition=greeting_workflow.to_dict(),
        api_key=API_KEY,
        parameters=params,
        stream=True
    ):
        try:
            event_data = json.loads(event)
            if 'output' in event_data and event_data['output'].strip():
                print(f"📤 {event_data['output'].strip()}")
        except:
            pass
    
    print("=" * 60)
    print("✅ Parameterized workflow completed!")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")


In [ ]:
## 3. Error Handling and Retry Logic

Workflows can handle failures gracefully with retry logic and conditional execution:


In [ ]:
# Create a workflow with error handling
resilient_workflow = workflow("error-handling-demo").description("Demonstrates retry and error handling")

# Step that might fail
check_step = step("check-service", """
    # Simulate a flaky service check
    if [ ! -f /tmp/kubiya_service_marker ]; then
        echo "❌ Service check failed, creating marker for retry..."
        touch /tmp/kubiya_service_marker
        exit 1
    else
        echo "✅ Service is ready!"
        rm -f /tmp/kubiya_service_marker
        exit 0
    fi
""")
check_step.retry(2, 1)  # Retry up to 2 times with 1 second delay

# Cleanup step that always runs
cleanup_step = step("cleanup", "echo '🧹 Performing cleanup operations...'")
cleanup_step.continue_on(failure=True)
cleanup_step.depends("check-service")

# Success notification (only runs if check succeeds)
notify_step = step("notify", "echo '📧 Sending success notification...'")
notify_step.depends("check-service")

# Add steps to workflow
resilient_workflow.data["steps"].extend([
    check_step.to_dict(),
    cleanup_step.to_dict(),
    notify_step.to_dict()
])

print("📋 Created workflow with:")
print("   ✅ Retry logic (2 attempts)")
print("   ✅ Cleanup that always runs")
print("   ✅ Conditional notification")


In [ ]:
# Execute workflow with error handling
print("\n🚀 Executing workflow with error handling...")
print("=" * 60)

retry_count = 0
outputs = []

try:
    for event in execute_workflow(
        workflow_definition=resilient_workflow.to_dict(),
        api_key=API_KEY,
        stream=True
    ):
        try:
            event_data = json.loads(event)
            
            # Show outputs
            if 'output' in event_data and event_data['output'].strip():
                output = event_data['output'].strip()
                outputs.append(output)
                print(f"📤 {output}")
            
            # Track retries
            elif 'retry' in str(event_data).lower():
                retry_count += 1
                print(f"🔄 Retry attempt {retry_count}")
                
        except:
            pass
    
    print("=" * 60)
    print(f"✅ Workflow completed!")
    print(f"   Retries: {retry_count}")
    print(f"   Total outputs: {len(outputs)}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")


In [ ]:
## Summary

In this notebook, you learned:

1. **Basic Workflows**: How to create and execute simple sequential workflows
2. **Parameters**: How to pass dynamic parameters to workflows
3. **Error Handling**: How to implement retry logic and conditional execution

### Key Takeaways:
- Workflows are composed of sequential steps
- Each step can be a shell command, Python script, or Docker container
- Parameters allow dynamic workflow behavior
- Error handling ensures robust execution
- The SDK automatically handles runner selection

### Next Steps:
- Explore **02_ai_workflow_generation.ipynb** for AI-powered workflow creation
- Check **08_workflow_orchestration.ipynb** for parallel execution patterns
- See **11_real_world_examples.ipynb** for production-ready workflows

### Using Local Runners:
While the SDK automatically selects appropriate runners, you can also set up local Kubernetes runners for:
- Enhanced security and compliance
- Custom resource requirements
- Private registry access
- Air-gapped environments

Refer to the documentation for local runner setup instructions.


In [ ]:
import os
from dotenv import load_dotenv
from kubiya_workflow_sdk import workflow, step, execute_workflow
from kubiya_workflow_sdk.execution import execute_workflow_logged, LogLevel

# Load environment variables from .env file
load_dotenv()

# Get API key
api_key = os.getenv("KUBIYA_API_KEY")
if not api_key:
    print("⚠️  Warning: KUBIYA_API_KEY not set")
    print("Please create a .env file with: KUBIYA_API_KEY='your-api-key'")
    raise ValueError("KUBIYA_API_KEY environment variable is required")

print(f"✅ API Key configured (length: {len(api_key)})")
print("📚 SDK loaded successfully!")
print("🔗 Using Kubiya API at: https://api.kubiya.ai")


In [ ]:
# Example 1: Basic Sequential Workflow
# This creates a simple workflow with two shell steps

hello_workflow = (
    workflow("hello-world")
    .description("A simple hello world workflow")
    .step("greeting", "echo 'Hello from Kubiya!'")
    .step("date", "date")
)

# Display the workflow definition
import json
print("📋 Workflow Definition:")
print(json.dumps(hello_workflow.to_dict(), indent=2))


In [ ]:
# Execute the workflow with enhanced logging
print("🚀 Executing workflow with real Kubiya API...")
print("=" * 60)

try:
    # Execute with logging for better visibility
    event_count = 0
    for event in execute_workflow_logged(
        workflow_definition=hello_workflow.to_dict(),
        api_key=api_key,
        log_level=LogLevel.NORMAL
    ):
        event_count += 1
        # The logging is handled by execute_workflow_logged
        
    if event_count > 0:
        print(f"\n✅ Successfully received {event_count} events")
    else:
        print("\n⚠️  No events received")
        
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("\nTroubleshooting:")
    print("1. Verify your API key is valid at https://app.kubiya.ai/settings/api-keys")
    print("2. Check network connectivity to api.kubiya.ai")
    print("3. Ensure your organization has active runners")


# Kubiya Workflow SDK - Getting Started

This notebook introduces the fundamental concepts of the Kubiya Workflow SDK, showing how to:
- Create basic workflows with sequential steps
- Execute workflows with proper error handling
- Use different executors (shell, python, docker)
- Handle workflow parameters and outputs
- Monitor execution with logging

## Prerequisites

Make sure you have:
1. Installed the SDK: `pip install kubiya-workflow-sdk`
2. Set your API key: `export KUBIYA_API_KEY="your-api-key"`

In [ ]:
import os
from kubiya_workflow_sdk import workflow, step, execute_workflow
from kubiya_workflow_sdk.execution import execute_workflow_logged, LogLevel

# Ensure API key is set
api_key = os.getenv("KUBIYA_API_KEY")
if not api_key:
    print("⚠️  Warning: KUBIYA_API_KEY not set")
    print("Please set it with: export KUBIYA_API_KEY='your-api-key'")
    print("Get your API key from: https://app.kubiya.ai/settings/api-keys")
    raise ValueError("KUBIYA_API_KEY environment variable is required")

print(f"✅ API Key configured (length: {len(api_key)})")
print("📚 SDK loaded successfully!")
print("🔗 Using Kubiya API at: https://api.kubiya.ai")

In [ ]:
# Example 1: Basic Sequential Workflow
# This creates a simple workflow with two shell steps

hello_workflow = (
    workflow("hello-world")
    .description("A simple hello world workflow")
    .step("greeting", "echo 'Hello from Kubiya!'")
    .step("date", "date")
)

# Display the workflow definition
import json
print("📋 Workflow Definition:")
print(json.dumps(hello_workflow.to_dict(), indent=2))